In [2]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_excel("Christmas Sales and Trends - December 2023.xlsx", index_col=0)
df.head()

,Date,Time,CustomerID,Age,Gender,Location,StoreID,OnlineOrderFlag,ProductID,ProductName,Category,Quantity,UnitPrice,TotalPrice,PaymentType,PromotionApplied,DiscountAmount,GiftWrap,ShippingMethod,DeliveryTime,Weather,Event,CustomerSatisfaction,ReturnFlag
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,
1,2020-12-24,07:27:59,441,27,Other,City_15,NaN,True,106,Toys_Product,Toys,5,96.786255,483.931273,Credit Card,False,0.0,False,Standard,5.0,Snowy,NaN,5,False
2,2022-11-18,14:36:39,340,43,Male,City_13,NaN,True,816,Clothing_Product,Clothing,1,95.279579,95.279579,Credit Card,True,0.0,True,Express,3.0,Sunny,NaN,2,True
3,2019-12-26,20:23:50,31,25,Other,City_7,92.0,False,508,Clothing_Product,Clothing,2,52.371645,104.743291,Credit Card,False,0.0,False,NaN,NaN,Rainy,Christmas Market,4,False
4,2018-11-13,23:08:08,39,64,Male,City_20,100.0,False,710,Toys_Product,Toys,5,63.647293,318.236463,Debit Card,False,0.0,True,NaN,NaN,Rainy,NaN,1,True
5,2020-12-13,04:38:08,344,26,Other,City_10,90.0,False,687,Toys_Product,Toys,3,57.384041,172.152122,Cash,False,0.0,True,NaN,NaN,Sunny,Christmas Market,4,False


In [4]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 10000 filas y 24 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 4 columnas con datos nulos, y son:
['StoreID', 'ShippingMethod', 'DeliveryTime', 'Event']
y sin nulos hay 20 columnas y son:
['Date', 'Time', 'CustomerID', 'Age', 'Gender', 'Location', 'OnlineOrderFlag', 'ProductID', 'ProductName', 'Category', 'Quantity', 'UnitPrice', 'TotalPrice', 'PaymentType', 'PromotionApplied', 'DiscountAmount', 'GiftWrap', 'Weather', 'CustomerSatisfaction', 'ReturnFlag']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Date,0.0%,100.0%,datetime64[ns],366
Time,0.0%,100.0%,object,9455
CustomerID,0.0%,100.0%,int64,500
Age,0.0%,100.0%,int64,53
Gender,0.0%,100.0%,object,3


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Time,10000,9455,21:21:38,4
Gender,10000,3,Male,3385
Location,10000,20,City_13,550
ProductName,10000,5,Electronics_Product,2053
Category,10000,5,Electronics,2053
PaymentType,10000,4,Cash,2540
ShippingMethod,5418,3,Express,1819
Weather,10000,3,Rainy,3408
Event,7162,2,Black Friday,4254


Principales estadísticos de las columnas numéricas:


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
Date,10000,NaN,NaN,NaN,2021-05-31 23:03:41.760000256,2018-11-01 00:00:00,2019-12-02 00:00:00,2021-11-01 00:00:00,2022-11-30 00:00:00,2023-12-31 00:00:00,NaN
CustomerID,10000.0,NaN,NaN,NaN,251.933,1.0,127.0,254.0,379.0,500.0,145.00392
Age,10000.0,NaN,NaN,NaN,43.7917,18.0,30.0,44.0,57.0,70.0,15.366585
StoreID,4582.0,NaN,NaN,NaN,50.920559,1.0,26.0,51.0,76.0,100.0,28.864546
OnlineOrderFlag,10000,2,True,5418,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ProductID,10000.0,NaN,NaN,NaN,500.2348,1.0,252.0,500.0,750.0,1000.0,288.741317
Quantity,10000.0,NaN,NaN,NaN,3.0106,1.0,2.0,3.0,4.0,5.0,1.404452
UnitPrice,10000.0,NaN,NaN,NaN,54.962118,10.003242,32.053314,55.035737,77.849652,99.996579,26.23937
TotalPrice,10000.0,NaN,NaN,NaN,165.426015,10.017153,71.897205,136.291203,241.049782,499.982894,116.107776
PromotionApplied,10000,2,True,5041,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Date,0.0%,100.0%,datetime64[ns],366
Time,0.0%,100.0%,object,9455
CustomerID,0.0%,100.0%,int64,500
Age,0.0%,100.0%,int64,53
Gender,0.0%,100.0%,object,3
Location,0.0%,100.0%,object,20
StoreID,54.18%,45.82%,float64,100
OnlineOrderFlag,0.0%,100.0%,bool,2
ProductID,0.0%,100.0%,int64,1000
ProductName,0.0%,100.0%,object,5


In [5]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['Time', 'Gender', 'Location', 'ProductName', 'Category', 'PaymentType', 'ShippingMethod', 'Weather', 'Event']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'TIME' -----------

Sus valores únicos son: [datetime.time(7, 27, 59) datetime.time(14, 36, 39)
 datetime.time(20, 23, 50) ... datetime.time(23, 20, 58)
 datetime.time(11, 33, 17) datetime.time(17, 1, 1)]

Las frecuencias de los valores únicos de las categorías son: Time
21:21:38    4
22:23:44    3
17:16:34    3
19:22:44    3
11:05:37    3
           ..
16:42:27    1
03:20:00    1
20:28:12    1
01:01:12    1
17:01:01    1
Name: count, Length: 9455, dtype: int64 
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'GENDER' -----------

Sus valores únicos son: ['Other' 'Male' 'Female']

Las frecuencias de los valores únicos de las categorías son: Gender
Male      3385
Other     3317
Female    3298
Name: count, dtype: int64 
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'LOCATION' -----------

Sus valores únicos son: ['City_15' 'City_13' 'City

In [6]:
def get_duplicate_rows(df):
    """
    Encuentra y devuelve las filas completas duplicadas de un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las filas duplicadas completas.
    """
    # Identificar duplicados basados en todas las columnas
    duplicate_rows = df[df.duplicated(keep=False)]
    return duplicate_rows

get_duplicate_rows(df)

,Date,Time,CustomerID,Age,Gender,Location,StoreID,OnlineOrderFlag,ProductID,ProductName,Category,Quantity,UnitPrice,TotalPrice,PaymentType,PromotionApplied,DiscountAmount,GiftWrap,ShippingMethod,DeliveryTime,Weather,Event,CustomerSatisfaction,ReturnFlag
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,
